In [3]:
import numpy as np
import os
import time
import ctypes
import sys

# from ctypes import *

# pylablib
ref: [thorlabs kinesis](https://pylablib.readthedocs.io/en/latest/devices/Thorlabs_kinesis.html#thorlabs-apt-kinesis-devices)

In [4]:
from typing import Any
import functools
from pylablib.devices import Thorlabs
import time


class FP_system_control(object):
    """
    Fourier Ptychography system control Class
    """

    def __init__(
        self,
        scale=(34555.0, 772970.0, 264.0),
        move_min_velocity=0,
        move_max_velocity=1,
        acceleration=1,
        home_direction="forward",
        *args,
        **kwargs,
    ) -> None:
        """
        initialize the whole system: translation stages and cameras
        """

        self.motor_SN = [
            device[0] for device in Thorlabs.list_kinesis_devices()
        ]  # translation stage serial number
        print(f"translation stage S/N: {self.motor_SN}")
        self.stage = {}  # translation stage objects
        # self.axis = [
        #     chr(97 + 23 + i) for i in range(len(self.motor_SN))
        # ]  # axis name 'x', 'y', and 'z'

        ## config translation stage parameters
        self.scale = scale  # device unit in [1 mm / scale]
        self.move_min_velocity = move_min_velocity
        self.move_max_velocity = move_max_velocity
        self.acceleration = acceleration
        self.home_direction = home_direction

        print("start to initialize all translation stages")
        self.init_all_stage()  # initialize all translation stages

    def __str__(self) -> str:
        """
        brief introduction
        """

        return "Fourier Ptychography camera system control"

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        """
        decorator
        """

        pass

    def init_all_stage(
        self,
    ) -> None:
        """
        initialize all translation stages
        """

        for SN in self.motor_SN:
            print(f"initialize translation stage {SN}")
            self.stage[SN] = Thorlabs.KinesisMotor(
                SN, self.scale
            )  # connect to stage and set scale. [1 mm / scale]

            ## config moving velocity
            self.stage[SN].setup_velocity(
                min_velocity=self.move_min_velocity,
                max_velocity=self.move_max_velocity,
                acceleration=self.acceleration,
            )

            ## config home mode
            self.stage[SN].setup_homing(
                velocity=self.move_max_velocity,
                home_direction=self.home_direction,
                limit_switch=self.home_direction,
            )

            ## config jog mode
            self.stage[SN].setup_jog(
                min_velocity=self.move_min_velocity,
                max_velocity=self.move_max_velocity,
                acceleration=self.acceleration,
            )

            self.stage[SN].home(force=True)  # home the stage
            self.stage[SN].wait_for_home()  # wait for "home"

        time.sleep(2)  # wait for the last translation stage to finish "home"

    def change_stage_name(self, old_name, new_name="x") -> None:
        """
        change translation stage name
        """

        self.stage[new_name] = self.stage.pop(old_name)

    def close_stages(self, close_all=True, name_list=[]) -> None:
        """
        turn off the connection to the translation stage(s)
        """
        if close_all:
            name_list = self.stage.keys()

        for name in name_list:
            self.stage[name].close

    def move_cam_xy(self, pos=[]) -> None:
        """
        move the camera(s) to a (x,y) position
        define the back view of camera(s) system
        y
        ^
        |
        |
        o------->x
        """

        pass

In [5]:
FP_xy_1cam = FP_system_control()

translation stage S/N: ['27257923', '27266045']


In [10]:
FP_xy_1cam.close_stages()

In [9]:
FP_xy_1cam.stage

{'27257923': <pylablib.devices.Thorlabs.kinesis.KinesisMotor at 0x274fd383d00>,
 '27266045': <pylablib.devices.Thorlabs.kinesis.KinesisMotor at 0x274fd39afe0>}

In [3]:
chr(97 + 23)

'x'

In [190]:
[device[0] for device in Thorlabs.list_kinesis_devices()]

['27266045', '27257923']

In [163]:
x_stage = Thorlabs.KinesisMotor(
    Thorlabs.list_kinesis_devices()[0][0], scale=(34555.0, 772970.0, 264.0)
)  # [1 mm / scale]


y_stage = Thorlabs.KinesisMotor(
    Thorlabs.list_kinesis_devices()[1][0], scale=(34555.0, 772970.0, 264.0)
)  # [1 mm / scale]


# modfied in class KinesisMotor(KinesisDevice):


# if stage in {"MTS25-Z8","MTS50-Z8"}:


#             return 34555E0,"mm"

In [164]:
x_stage.setup_velocity(min_velocity=0, max_velocity=1, acceleration=1)
x_stage.setup_homing(velocity=1, home_direction="forward", limit_switch="forward")
x_stage.setup_jog(min_velocity=0, max_velocity=1, acceleration=1)

y_stage.setup_velocity(min_velocity=0, max_velocity=1, acceleration=1)
y_stage.setup_homing(velocity=1, home_direction="forward", limit_switch="forward")
y_stage.setup_jog(min_velocity=0, max_velocity=1, acceleration=1)

TJogParams(mode='step', step_size=0.4999855303139922, min_velocity=0.0, acceleration=1.0, max_velocity=1.0, stop_mode='profiled')

In [165]:
print(x_stage.get_full_info())
print(y_stage.get_full_info())

{'velocity_parameters': TVelocityParams(min_velocity=0.0, acceleration=1.0, max_velocity=1.0), 'jog_parameters': TJogParams(mode='step', step_size=0.4999855303139922, min_velocity=0.0, acceleration=1.0, max_velocity=1.0, stop_mode='profiled'), 'homing_parameters': THomeParams(home_direction='forward', limit_switch='forward', velocity=1.0, offset_distance=1.0), 'gen_move_parameters': TGenMoveParams(backlash_distance=0.05000723484300391), 'limit_switch_parameters': TLimitSwitchParams(hw_kind_cw='make', hw_kind_ccw='make', hw_swapped=False, sw_position_cw=3.0, sw_position_ccw=1.0, sw_kind='ignore'), 'kcube_trigio_parameters': TKCubeTrigIOParams(trig1_mode='off', trig1_pol=True, trig2_mode='off', trig2_pol=True), 'kcube_trigpos_parameters': TKCubeTrigPosParams(start_fw=0.0, step_fw=0.0, num_fw=1, start_bk=0.0, step_bk=0.0, num_bk=1, width=0.05, ncycles=1), 'position': 0.0, 'status': ['sw_bk_lim', 'connected', 'homed', 'digio1', 'digio2', 'power_ok', 'enabled'], 'cls': 'KinesisMotor', 'conn

In [184]:
x_stage.home(force=True)  # home the stage
y_stage.home(force=True)  # home the stage

x_stage.wait_for_home()  # wait until homing is done
y_stage.wait_for_home()  # wait until homing is done

In [183]:
print(x_stage.get_position())
print(y_stage.get_position())

-25.000086818116046
-25.0


In [185]:
x_stage.move_to(-25)
x_stage.wait_move()
y_stage.move_to(-25)
y_stage.wait_move()

In [204]:
x_stage.close()
y_stage.close()

# thorlabs-kinesis

In [2]:
if sys.version_info < (3, 8):
    os.chdir(r"C:\Program Files\Thorlabs\Kinesis")
else:
    os.add_dll_directory(r"C:\Program Files\Thorlabs\Kinesis")

lib: CDLL = cdll.LoadLibrary("Thorlabs.MotionControl.KCube.DCServo.dll")

In [3]:
# Uncomment this line if you are using simulations
lib.TLI_InitializeSimulations()

# Set constants
serial_num = c_char_p(b"27000001")

if lib.TLI_BuildDeviceList() == 0:
    lib.CC_Open(serial_num)
    lib.CC_StartPolling(serial_num, c_int(200))

    if lib.CC_CanHome(serial_num):
        lib.CC_Home(serial_num)

        lib.CC_RequestStatusBits(serial_num)
        print(lib.CC_GetStatusBits(serial_num))

        while lib.CC_GetPositionCounter(serial_num) != 0:
            pass

        time.sleep(0.1)

        print("home operation completed!")

        lib.CC_RequestStatusBits(serial_num)
        print(lib.CC_GetStatusBits(serial_num))

    # # Set up the device to convert real units to device units
    # STEPS_PER_REV = c_double(34555)  # for the MTS50-Z8
    # gbox_ratio = c_double(1.0)  # gearbox ratio
    # pitch = c_double(1.0)

    # # Apply these values to the device
    # lib.CC_SetMotorParamsExt(serial_num, STEPS_PER_REV, gbox_ratio, pitch)

    # # Get the device's current position in dev units
    # lib.CC_RequestPosition(serial_num)
    # time.sleep(0.2)
    # dev_pos = c_int(lib.CC_GetPosition(serial_num))

    lib.CC_Close(serial_num)


# Uncomment this line if you are using simulations
lib.TLI_UninitializeSimulations()

0
home operation completed!
-2147482624


2

# thorlabs_apt_device

In [2]:
from thorlabs_apt_device import TDC001, KDC101

In [30]:
stage = TDC001("COM6", home=False)
time.sleep(1)
while (
    stage.status["moving_forward"] == True
    or stage.status["moving_reverse"] == True
    or stage.status["homing"] == True
):
    pass

In [34]:
stage.home()
time.sleep(1)
while (
    stage.status["moving_forward"] == True
    or stage.status["moving_reverse"] == True
    or stage.status["homing"] == True
):
    pass

In [227]:
stage.stop(True)

In [35]:
stage.status

{'position': 0,
 'enc_count': 0,
 'velocity': 0,
 'forward_limit_switch': False,
 'reverse_limit_switch': False,
 'moving_forward': False,
 'moving_reverse': False,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': True,
 'homing': False,
 'homed': True,
 'tracking': False,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_dcstatusupdate',
 'msgid': 1169,
 'source': 80,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': True,
 'reverse_limit_soft': False,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': False,
 'digital_in_2': False,
 'digital_in_3': False,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': True,
 'active': False,
 'error': False}

In [32]:
# Flash the LED on the device to identify it
stage.identify()
stage.set_velocity_params(264, 772970)

stage.move_absolute(34555 * 10)
time.sleep(1)
while (
    stage.status["moving_forward"] == True
    or stage.status["moving_reverse"] == True
    or stage.status["homing"] == True
):
    pass
time.sleep(1)
# Do some moves (encoder counts)
# or
# stage.move_absolute(12345)

In [33]:
# See the position (in encoder counts)
print(stage.status["position"])

345550


In [23]:
stage.status["moving_forward"]

False

In [27]:
stage.close()